# Quantum States and Gates Exploration

This notebook tests the functionality of quantum statevector simulators and quantum gate simulators.

In [38]:
import numpy as np
import numpy.typing as npt
from random import random

## Quantum States

Quantum states should be able to:
1. Store complex amplitudes
3. Support measurement
4. Handle multi-qubit systems
5. Evolve according to quantum operations

### Utilities

The absolute value of a number squared will likely be used often to find the probabilities of measurement outcomes.

The Euclidean norm of a vector will likely be used often to check the validity of a statevector or a unitary matrix.

The length of a statevector must be equal to a power of 2

In [39]:
def abs_squared(num: complex | float):
    return np.abs(np.square(num))

def e_norm(vector: npt.NDArray):
    return np.sum(abs_squared(vector))

def is_pow_of_2(length: int):
    while length % 2 == 0:
        length //= 2
    return length == 1

### Version 1

The `Qubit` class will keep track of individual qubits, including their initialization, measurements, probabilities, and evolutions.

In [40]:
class Qubit:
    def __init__(self, alpha: complex, beta: complex):
        if e_norm([alpha, beta]) != 1:
            print(e_norm([alpha, beta]))
            raise Exception("The Euclidean norm of the state must be 1")
        
        self.alpha = alpha
        self.beta = beta

    def __str__(self):
        return f"{self.alpha}|0⟩ + {self.beta}|1⟩"
    
    def measure(self):
        if random() < abs_squared(self.alpha):
            self.alpha = 1
            self.beta = 0
            return 0
        else:
            self.alpha = 0
            self.beta = 1
            return 1
        
    def evolve(self, operator: npt.NDArray):
        if operator.shape != (2, 2):
            raise Exception(f"The operator's shape {operator.shape} is incompatible with the qubit")
        state = np.ndarray([self.alpha, self.beta])
        transformed = operator @ state
        self.alpha = transformed[0]
        self.beta = transformed[1]

    def to_numpy(self):
        return np.ndarray([self.alpha, self.beta])
        

### Testing

Testing reveals that `e_norm` may result in rounding errors 

In [41]:
# This test fails because the Euclidean norm is not exactly 1
# q = Qubit(1/np.sqrt(2), 1/np.sqrt(2))

### Version 2

Add tolerance to Euclidean norm checks and rounding to string representation

In [42]:
class Qubit:
    def __init__(self, alpha: complex, beta: complex):
        if abs(1 - e_norm([alpha, beta])) > 1e-5:
            raise Exception("The Euclidean norm of the state must be 1")
        
        self.alpha = alpha
        self.beta = beta

    def __str__(self):
        return f"{self.alpha:.4f}|0⟩ + {self.beta:.4f}|1⟩"
    
    def measure(self):
        if random() < abs_squared(self.alpha):
            self.alpha = 1
            self.beta = 0
            return 0
        else:
            self.alpha = 0
            self.beta = 1
            return 1
        
    def evolve(self, operator: npt.NDArray):
        if operator.shape != (2, 2):
            raise Exception(f"The operator's shape {operator.shape} is incompatible with the qubit")
        state = np.ndarray([self.alpha, self.beta])
        transformed = operator @ state
        self.alpha = transformed[0]
        self.beta = transformed[1]

    def to_numpy(self):
        return np.ndarray([self.alpha, self.beta])
        

### Testing

These tests show that the initialization, measurement, and string representation functions are working properly

In [61]:
q = Qubit(1/np.sqrt(2), 1/np.sqrt(2))
print(q)

q = Qubit(1, 0)
print(q)

q = Qubit(0.5, complex(0, np.sqrt(3) / 2))
print(q)

print(q.measure())
print(q)

count = {
    0: 0,
    1: 0
}
for i in range(1000):
    q = Qubit(0.5, complex(0, np.sqrt(3) / 2))
    measurement = q.measure()
    count[measurement] += 1

# 0 should be measured 25% of the time
# 1 should be measured 75% of the time
print(count)

0.7071|0⟩ + 0.7071|1⟩
1.0000|0⟩ + 0.0000|1⟩
0.5000|0⟩ + 0.0000+0.8660j|1⟩
1
0.0000|0⟩ + 1.0000|1⟩
{0: 257, 1: 743}
